<a href="https://colab.research.google.com/github/ravmarcin/GoogleColab/blob/main/mentor2mentee.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
user = 'Oleksandra Tsymbal'
user_profile = 'https://www.linkedin.com/in/oleksandramarciniak/'

# 1. Install required libraries.

In [2]:
!pip install googletrans==3.1.0a0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# 2. Mount the drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 3. Import functions, libraries and directories. Prefilter Mentee for non assign

In [4]:
import sys
import os
import nltk
from google.colab import data_table

home_dir = '/content/drive/MyDrive/Colab Notebooks/'
sys.path.insert(0, home_dir + 'functions/')
from utils import *
from widgets import *

#Import nltk objects
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

# File location
mentor_pd = home_dir + 'Data/Goodjob Mentorship Program _ Mentor Form (Responses).xlsx'
mentee_pd = home_dir + 'Data/Goodjob Mentorship Program _ Mentee Form (Responses).xlsx'
mentee_pd_temp = home_dir + 'temp/Goodjob Mentorship Program _ Mentee Form (Responses).xlsx'
lang_file = home_dir + '/Utils/Languages.txt'

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
# assign column = 14
check_df = check_if_assign(df_dir=mentee_pd, col_check=14)
# column for id check
# new_df = check_if_new_rec(new_ver_dir=mentee_pd, temp_dir=mentee_pd_temp, col_check=1)

# [language, new role, theme, location]
params_for_mentee = [5, 9, 10, 12]
mentee_df_new_reduce = reduce_df(df=check_df, params=params_for_mentee, id_col=1)


# [language, current role, theme, location]
params_for_mentor = [4, 8, 10, 13]
mentor_df = import_xlsx(mentor_pd)
mentor_df_reduce = reduce_df(df=mentor_df, params=params_for_mentor, id_col=1)


# trans mentee with these new columns and get one mentee
cols_to_trans = [1, 2, 3, 4]
mentee_df_new_reduce_trans = trans_df(df=mentee_df_new_reduce, lang='en', cols=cols_to_trans, trans_col_names=False)

# 4. Select Mentee

In [6]:
# Select Mentee
drop_mentee = IActiveDF(df_in=check_df)


interactive(children=(Dropdown(description='E-mails', options=('irooon.pop@gmail.com', 'anna.chubatiuk13@gmail…

# 5. Match Mentor to Mentee

In [7]:
# Get selected dataframe
mentee_df_new_reduce_trans_one = get_mentee(df=mentee_df_new_reduce_trans, val=drop_mentee.value, col=0)

# Filtering the languages for Mentors
lang_mentor_ls_uq, lang_mentor_np, mentor_df_reduce_fil = filter_by(df=mentor_df_reduce, col=1, f_dir=lang_file)


# Matching Mentors to Mentees based on the language
lang_strong_match_ls = strong_match(df_to_fil=mentee_df_new_reduce_trans_one,
                                    df_from_to=mentor_df_reduce_fil, f_dir=lang_file,
                                    words_uq=lang_mentor_ls_uq, words_ls=lang_mentor_np,
                                    by_col_to=1, col_to=0, col_from=0)


# Matching Mentors to Mentees based on the text
text_light_match_ls = light_match(df_to_fil=mentee_df_new_reduce_trans_one,
                                  df_from_to_ls=lang_strong_match_ls, by_cols_to=[3], by_cols_from=[3])



# 6. Select mentor

In [8]:
columns_vis = ['Email', 'Language', 'New role', 'What to learn', 'Location']
mentee_vis = mentee_df_new_reduce_trans_one
mentee_vis.columns = columns_vis
data_table.DataTable(mentee_df_new_reduce_trans_one, include_index=False) 

,Email,Language,New role,What to learn,Location
1,irooon.pop@gmail.com,"Ukrainian, Russian, English",content creator\ art director\ project manager,CV\ interview\ how to introduce yourself witho...,Ukraine


In [9]:
# View the table
mentor_vis = text_light_match_ls[0]
mentor_vis.columns = columns_vis + ['Ratio']
col_ls = mentor_vis.columns.values.tolist()
data_table.DataTable(mentor_vis.sort_values(by=col_ls[-1], ascending=False), include_index=False)

,Email,Language,New role,What to learn,Location,Ratio
125,andrearosena@gmail.com,english hebrew,Business Development,- cv review\n- business development roles\n- l...,"Currently based in Berlin, Germany. Lived pre...",0.153846
36,oksana.rysovana@gmail.com,ukrainian russian english,Co-founder & COO,"- how to find work in fashion (CV review, wher...","Currently in Manresa, Spain, lived in Kyiv bef...",0.142857
135,matthew.simms@resourcesolutions.com,english,Direct Recruiter,"Job searching, CV prep, interview prep, Accoun...",Previously Robert Walters,0.142857
73,oksana.rysovana@gmail.com,ukrainian russian english,Co-founder & COO,"- how to find work in fashion (CV review, wher...","Currently in Manresa, Spain, lived in Kyiv bef...",0.142857
65,rafael.s.rosenstein@icloud.com,german english,Account strategist,"Interview; CV; networking, coaching","Germany, dusseldorf",0.142857
...,...,...,...,...,...,...
104,elizagrigorieva@gmail.com,ukrainian russian english,social media manager analist,"social media marketing, branding, desigh ideas","Spain, Alicante, Ukraine Odessa",0.000000
62,Rkentie12@gmail.com,english dutch,Analyst,Whatever you need. I can help with interview P...,Dublin,0.000000
78,iryna@irynashostak.com,ukrainian russian english,Wedding photographer,Wedding photography skills and business,"NJ, USA, from Poltava",0.000000
109,Lnitisa@gmail.com,latvian english russian,Tax adviser,Tax related questions regaring Latvia - both i...,"Riga, Latvia",0.000000


In [10]:
drop_mentor = IActiveDF(df_in=text_light_match_ls[0], col=0, by_col=-1)

interactive(children=(Dropdown(description='E-mails', options=('andrearosena@gmail.com', 'oksana.rysovana@gmai…

# 7. Update the file.

In [11]:
mentee_df = import_xlsx(in_dir=mentee_pd)
mentee_df_update = update_df(df=mentee_df, val_idx=drop_mentee.value, val_to_fill=drop_mentor.value, user=user)
# Hashtag this line for automatic run
# save_xlsx(df=mentee_df_update, in_dir=mentee_pd, out_dir=mentee_pd)

# 8. Getting the info for email.

In [25]:
mentee_dict = get_mentee_info(df=mentee_df_update, val=str(drop_mentee.value))
mentee_name = str(mentee_dict['name']).split()[0]
mentor_dict = get_mentor_info(df=mentor_df, val=str(drop_mentor.value))
mentor_name = str(mentor_dict['name']).split()[0]
user_name = str(user).split()[0]

In [35]:
class style:
   BOLD = '\033[1m'
   END = '\033[0m'
   ITALIC = '\x1B[3m'
   I_END = '\x1B[0m'

email = [
         f"Hi " + style.BOLD + mentee_name + style.END + f" and " + style.BOLD + mentor_name + style.END + f"\n\n",
         f"My name is " + style.BOLD + user_name + style.END + f" (" + user_profile + ").\n",
         f"I’m a Mentorship Coordinator at https://www.findyourgoodjob.com/. And I hope you are both doing well! \n",
         f"Dear " + style.BOLD + mentee_name + style.END + f", I’m happy to introduce you to – " + style.BOLD + mentor_name + style.END + f", who will be your mentor. \n",
		     f"Huge thank you to you, " + style.BOLD + mentor_name + style.END + f", for offering your mentorship to Ukrainians. We appreciate your willingness to help from the bottom of our hearts!",
		     f"\U0001F493",
		     f"\n\n\n",
		     f"About " + style.BOLD + mentor_name + style.END + f":\n",
		     mentor_dict['role'] + f"\n",
		     style.BOLD + f"LinkedIn profile: " + style.END + mentor_dict['linkedin'] + f"\n",
		     style.BOLD + f"Location: " + style.END  + mentor_dict['location'] + f"\n",
		     style.BOLD + f"Experience: " + style.END  + mentor_dict['experience'] + f"\n",
		     style.BOLD + f"Happy to help with: " + style.END  + mentor_dict['topic'] + f"\n",
		     f"\n\n\n",
		     f"About " + style.BOLD + mentee_name + style.END + f":\n",
		     mentee_dict['role'] + f"\n",
		     style.BOLD + f"LinkedIn profile: " + style.END + mentee_dict['linkedin'] + f"\n",
		     style.BOLD + f"Location: " + style.END  + mentee_dict['location'] + f"\n",
		     style.BOLD + f"Experience in: " + style.END  + mentee_dict['experience'] + f"\n",
		     style.BOLD + f"Goal: " + style.END  + mentee_dict['topic'] + f"\n",
         f"\n\n\n",
         style.BOLD + f"My role in this program is" + style.END  + f" – to help mentees find a perfect match (their mentor), and to introduce you to each other. From this point on, \n",
         style.BOLD + mentee_name + style.END + f", it's up to you to organize your first session with " + style.BOLD + mentor_name + style.END + f". \n",
         f"I would highly recommend sharing your CV and your goals before the first session.",
         f"\n\n\n",
         "Please keep our email – nastya@findyourgoojob.com in CC of your correspondence.",
         f"\n\n\n",
         style.BOLD + f"IMPORTANT:" + style.END,
         f"Helpful guidelines & expectations:\n\n",
         f"     1. Both mentor and mentee are " + style.BOLD +  f"bound by a confidentiality agreement" + style.END + f". Everything that is discussed during the sessions remains confidential, including any meeting notes.\n",
         f"     2. Both mentor and mentee are " + style.BOLD +  f"committed" + style.END + f" to being present and in sync to achieve the mentoring goals of their sessions. It is their " + style.BOLD +  f"mutual responsibility" + style.END + f" to show up for meetings and agree on their frequency based on their availability and needs.\n".rjust(20, ' '),
         f"     3. Originally your mentor is responsible for 2-3 sessions and is up to a mutual agreement to continue with regular sessions.\n",
         f"     4. It is the mentee’s responsibility to have an idea of the goals they would like to achieve through their sessions. It is a mutual responsibility of mentor and mentee to " + style.BOLD +  f"agree and clearly state the goals" + style.END + f" and ideal outcomes (state the metrics of success).\n".rjust(20, ' '),
         f"     5. It is the mentee’s responsibility to reach out to " + style.BOLD +  f"organize the meetings, and prepare and drive the conversations" + style.END + f" according to their goals. It is the mentor’s responsibility to guide the mentee and answer any questions or bring them in contact with other team members if needed.\n".rjust(20, ' '),
         f"     6. It is the mentee’s responsibility to " + style.BOLD +  f"“do the homework”" + style.END + f", and show commitment to achieving the set goals.\n",
         f"\n\n\n",
         style.ITALIC + style.BOLD + f"Worth Mentioning:\n\n" + style.END + style.I_END,
         style.ITALIC + f"The goal of this initiative is to bring Ukrainian mentees closer to successful mentors from all over the world and help Ukrainians to ease up the burden of finding a new job or making the next career step during those difficult times.\n\n" + style.I_END,
         style.ITALIC + f"It will help if you are clear-minded about what your goal is before speaking with your mentor. The conversation may take a different path, but you’ll generally get more out of the chat if you can pinpoint your most important questions from the get-go.\n\n" + style.I_END,
         style.ITALIC + f"Mentors are here to help you navigate the complexity of evolving as a creator, and the challenges and opportunities that go with that. They are here to accelerate the path to your personal goals through sharing their own experience and unveiling implicit rules.\n\n" + style.I_END,
         style.ITALIC + f"They are not responsible for your results or performance, but may help you deliver on them faster!\n\n" + style.I_END,
         style.ITALIC + f"Your mentoring sessions will likely be one hour, bi-weekly or once a month, but there is not a set schedule, rather you will work with your mentor to find the best timing and frequency.\n\n" + style.I_END,
         style.ITALIC + f"Lastly, the content of your conversation should be treated as confidential. This means you shouldn’t share personal information your mentor may tell you about themselves, unless they tell you otherwise.\n\n" + style.I_END,
         f"Please let me know if you have any questions. I hope you will find this initiative helpful, fulfilling and rewarding.",
         f"\n\n\n",
         f"Thank you <3\n\n",
         f"Yours, " + style.ITALIC + user_name + style.I_END + f"\n",
         f"Mentorship Coordinator at https://www.findyourgoodjob.com/"
		     ]
text = ""
for line in email:
  text += line
print(text)

Hi Andrii and Andrea

My name is Oleksandra (https://www.linkedin.com/in/oleksandramarciniak/).
I’m a Mentorship Coordinator at https://www.findyourgoodjob.com/. And I hope you are both doing well! 
Dear Andrii, I’m happy to introduce you to – Andrea, who will be your mentor. 
Huge thank you to you, Andrea, for offering your mentorship to Ukrainians. We appreciate your willingness to help from the bottom of our hearts!💓


About Andrea:
Business Development
LinkedIn profile: https://www.linkedin.com/in/rosenandrea/
Location: I agree
Expirance: I am a global development expert in the field of startup technology that focuses on impact (sustainable and social in urban environments).  I have expertise in business development, partnerships, market expansion, and product development. I have worked in the deep tech sector, sharing economy, agtech, marketplaces, and music and hardware technology.
Happy to help with: - cv review
- business development roles
- landing an international job
- jobs 